In [ ]:
import os
import sys
from copy import deepcopy

import numpy as np
from matplotlib import pyplot as plt

from hera_cal.io import HERACal, HERAData
from hera_cal.noise import interleaved_noise_variance_estimate
from hera_cal.utils import join_bl, split_bl

sys.path.insert(0, os.path.dirname(os.getcwd()))
from red_utils import find_zen_file

# Noise estimates from the hera_cal pipeline

Using calibrated autocorrelations, we can predict the noise variance on visibilities, $\sigma_{ij}^2$. Namely,

$\sigma_{ij}^2 = V_{ii} V_{jj}$ $ / $ $B t$

where $B$ is the bandwidth of a channel and $t$ is the integration time. Instead of computing this quantity for all baselines, we instead compute and save $\sigma_{ii}$ where

$\sigma_{ij} \equiv \sqrt{\sigma_{ii} \sigma_{jj}} = \left(V_{ii} / \sqrt{Bt}\right) \left( V_{jj} / \sqrt{Bt} \right)$.

These quantities, $\sigma_{ii}$, are stored in `.noise_std.uvh5` files. Though they are technically per-antenna, we the collaboration felt it more sensible to store them as visibility data files (since the units are Jy) with autocorrelation keys instead of storing them in `.calfits` files.

In [ ]:
JD_time = 2458098.43869
bl = (25, 51, 'ee')
data_dir = '/Users/matyasmolnar/HERA_Data/sample_data'

## Noise from raw autocorrelations

In [ ]:
ant1, ant2 = split_bl(bl)
auto_bl1 = join_bl(ant1, ant1)
auto_bl2 = join_bl(ant2, ant2)

# Load autocorrelation
autos_file = os.path.join(data_dir, 'zen.{}.HH.autos.uvh5'.format(JD_time))
hd_autos = HERAData(autos_file)
autos, auto_flags, _  = hd_autos.read(bls=[auto_bl1, auto_bl2])

# Load inferred noise on data
noise_file = os.path.join(data_dir, 'zen.{}.HH.noise_std.uvh5'.format(JD_time))
hd_noise = HERAData(noise_file)
noise, noise_flags, _  = hd_noise.read(bls=[auto_bl1, auto_bl2])
bl_noise = np.sqrt(noise[auto_bl1] * noise[auto_bl2])
bl_noise_flags = noise_flags[auto_bl1] | noise_flags[auto_bl2]

## Noise from interleaved frequencies

We can also check our inferred value for the noise on visibilities by checking them against a sequential difference of the data. In this case, we use hera_cal.noise.interleaved_noise_variance_estimate() to estimate the noise on the data by subtracting 0.5 times the next and previous channels from the data. Averaging in time over the file, we see that these two estimates of the noise agree.

In [ ]:
hd = HERAData(find_zen_file(JD_time))
data, flags, nsamples = hd.read(bls=[bl])

In [ ]:
# Estimate noise from visibility data using interleaved frequencies
data_with_nans = deepcopy(data[bl])
data_with_nans[flags[bl]] = np.nan
noise_var_est = interleaved_noise_variance_estimate(data_with_nans, kernel=[[-.5, 1, -.5]])
with np.errstate(divide='ignore', invalid='ignore'):
    interleaved_noise = np.sqrt(np.nanmean(noise_var_est, axis=0))

# Estimate noise on baseline using autocorrelations
var_with_nans = noise[auto_bl1] * noise[auto_bl2]
var_with_nans[flags[bl]] = np.nan
autocorrelation_noise = np.sqrt(np.abs(np.nanmean(var_with_nans, axis=0)))

noise_amp = np.nanmean((autocorrelation_noise / interleaved_noise)[420:900]) # good freq range

# Plot Results
fig, ax1 = plt.subplots(figsize=(11, 7))
ax1.plot(hd.freqs / 1e6, interleaved_noise * noise_amp, label='Interleaved Noise Estimate', lw=2)
ax2 = ax1.twinx()
ax2.plot(hd.freqs / 1e6, autocorrelation_noise, label='Noise Inferred from '\
         'Autocorrelations', color='orange', lw=2)

plt.xlim(100, 200)
plt.xlabel('Frequency (MHz)')
ax2.set_ylabel('Amplitude (Jy)')

ax1.set_ylim(bottom=0, top=50)
ax2.set_ylim(bottom=0, top=20)

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
lines = lines_1 + lines_2
labels = labels_1 + labels_2
ax1.legend(lines, labels, loc=0)

plt.show()